In [156]:
%matplotlib inline

In [157]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:.6f}'.format

# 데이터 불러오기

In [158]:
train = pd.read_csv('./Dataset/train.csv')
test = pd.read_csv('./Dataset/test.csv')
car_2020 = pd.read_csv('./Dataset/car_2020.csv',encoding='cp949')
submission = pd.read_csv('./Dataset/sample_submission.csv')

In [159]:
test.isnull().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              2
임대보증금                           180
임대료                             180
도보 10분거리 내 지하철역 수(환승노선 수 반영)     42
도보 10분거리 내 버스정류장 수                0
단지내주차면수                           0
dtype: int64

car_2020 = car_2020.drop(['시군구(1)'], axis=1)
car_2020.columns = ['지역','월','항목','분류','자동차수']
car_2020['월'] = pd.to_datetime(car_2020['월']).dt.month
car_2020.head()

In [160]:
train.isnull().sum()

단지코드                              0
총세대수                              0
임대건물구분                            0
지역                                0
공급유형                              0
전용면적                              0
전용면적별세대수                          0
공가수                               0
자격유형                              0
임대보증금                           569
임대료                             569
도보 10분거리 내 지하철역 수(환승노선 수 반영)    211
도보 10분거리 내 버스정류장 수                4
단지내주차면수                           0
등록차량수                             0
dtype: int64

# 컬렴명 바꿔주기

In [161]:
train.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '신분',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수', '등록차량수'
]


# 지하철 Y or N로 변경

In [162]:
train['지하철'] = train['지하철'].fillna(0)
train['지하철'] = np.where(train['지하철'] == 0, 0, 1)

# 결측치 처리

In [163]:
train.loc[train.임대보증금=='-', '임대보증금'] = np.nan
train.loc[train.임대료=='-', '임대료'] = np.nan
train['임대보증금'] = train['임대보증금'].astype(float)
train['임대료'] = train['임대료'].astype(float)
train['임대보증금'] = train['임대보증금'].fillna(0)
train['임대료'] = train['임대료'].fillna(0)

In [164]:
train['버스'] = train['버스'].fillna(train['버스'].mean())

In [165]:
train.isnull().sum()

단지코드        0
총세대수        0
임대건물구분      0
지역          0
공급유형        0
전용면적        0
전용면적별세대수    0
공가수         0
신분          0
임대보증금       0
임대료         0
지하철         0
버스          0
단지내주차면수     0
등록차량수       0
dtype: int64

# 임대건물구분 : 아파트->상가 / 전용면적별세대수 : x->1

In [166]:
idx = train[(train['임대건물구분']=='아파트') & (train['신분']=='D')]['전용면적별세대수'].index
train.loc[idx, '전용면적별세대수'] = 1
train.loc[idx, '임대건물구분'] = '상가'

In [167]:
train[(train['임대건물구분']=='상가') & (train['신분']=='D')]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,신분,임대보증금,임대료,지하철,버스,단지내주차면수,등록차량수
88,C1925,601,상가,강원도,임대상가,32.100000,1,9.000000,D,0.000000,0.000000,0,4.000000,117.000000,75.000000
89,C1925,601,상가,강원도,임대상가,32.100000,1,9.000000,D,0.000000,0.000000,0,4.000000,117.000000,75.000000
90,C1925,601,상가,강원도,임대상가,32.100000,1,9.000000,D,0.000000,0.000000,0,4.000000,117.000000,75.000000
91,C1925,601,상가,강원도,임대상가,72.160000,1,9.000000,D,0.000000,0.000000,0,4.000000,117.000000,75.000000
101,C1874,619,상가,충청남도,임대상가,12.620000,1,2.000000,D,0.000000,0.000000,0,2.000000,97.000000,62.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2333,C1350,1401,상가,대전광역시,공공분양,74.940000,1,2.000000,D,0.000000,0.000000,0,6.000000,1636.000000,2315.000000
2334,C1350,1401,상가,대전광역시,공공분양,84.940000,1,2.000000,D,0.000000,0.000000,0,6.000000,1636.000000,2315.000000
2335,C1350,1401,상가,대전광역시,공공분양,84.940000,1,2.000000,D,0.000000,0.000000,0,6.000000,1636.000000,2315.000000
2336,C1350,1401,상가,대전광역시,공공분양,84.960000,1,2.000000,D,0.000000,0.000000,0,6.000000,1636.000000,2315.000000


In [168]:
# 원핫 인코딩
#train = pd.get_dummies(data = train, columns = ['지하철'])

# 지역명 숫자로 매핑

In [169]:
local_map = {}
for i, loc in enumerate(train['지역'].unique()):
    local_map[loc] = i
    
train['지역'] = train['지역'].map(local_map)

# 세종 : 15

# 전용면적을 5의 배수로 변경

In [170]:
train['전용면적'] = train['전용면적']//5*5

# 전용면적 상/하한 적용

In [171]:
idx = train[train['전용면적']>100].index
train.loc[idx, '전용면적'] = 100

idx = train[train['전용면적']<15].index
train.loc[idx, '전용면적'] = 15

# 세종 데이터 분리

In [172]:
train_sejong = train[train['지역']==15]
train_other = train[train['지역']!=15]

# test 불러오기

In [173]:
test.columns = [
    '단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수', '신분',
    '임대보증금', '임대료', '지하철', '버스',
    '단지내주차면수'
]

In [174]:
test.isnull().sum()

단지코드          0
총세대수          0
임대건물구분        0
지역            0
공급유형          0
전용면적          0
전용면적별세대수      0
공가수           0
신분            2
임대보증금       180
임대료         180
지하철          42
버스            0
단지내주차면수       0
dtype: int64

In [175]:
test.loc[test.임대보증금=='-', '임대보증금'] = np.nan
test.loc[test.임대료=='-', '임대료'] = np.nan
test['임대보증금'] = test['임대보증금'].fillna(0)
test['임대료'] = test['임대료'].fillna(0)
test['임대보증금'] = test['임대보증금'].astype(float)
test['임대료'] = test['임대료'].astype(float)

In [176]:
test['지하철'] = test['지하철'].fillna(0)
test['지하철'] = np.where(test['지하철'] == 0, 0, 1)

In [177]:
test.loc[test.단지코드.isin(['C2411']) & test.신분.isnull(), '신분'] = 'A'
test.loc[test.단지코드.isin(['C2253']) & test.신분.isnull(), '신분'] = 'C'

In [178]:
local_map = {}
for i, loc in enumerate(test['지역'].unique()):
    local_map[loc] = i
    
test['지역'] = test['지역'].map(local_map)

In [179]:
test['전용면적'] = test['전용면적']//5*5

idx = test[test['전용면적']>100].index
test.loc[idx, '전용면적'] = 100

idx = test[test['전용면적']<15].index
test.loc[idx, '전용면적'] = 15

# 세종 데이터 분리

In [180]:
test_sejong = test[test['지역']==14]
test_other = test[test['지역']!=14]

In [181]:
test.isnull().sum()

단지코드        0
총세대수        0
임대건물구분      0
지역          0
공급유형        0
전용면적        0
전용면적별세대수    0
공가수         0
신분          0
임대보증금       0
임대료         0
지하철         0
버스          0
단지내주차면수     0
dtype: int64

In [182]:
columns = ['단지코드', '총세대수', '지역', '전용면적별세대수', '공가수', '지하철', '버스', '단지내주차면수']
target = '등록차량수'
area_columns = []
for area in train['전용면적'].unique():
    area_columns.append(f'면적_{area}')

In [183]:
new_train = pd.DataFrame()
new_test = pd.DataFrame()

In [184]:
new_train['임대보증금'] = 0
new_train['임대료'] = 0
new_test['임대보증금'] = 0
new_test['임대료'] = 0

# 수정한 코드

In [185]:
# Train Dataframe Set
# 미사용 열s - 임대건물구분, 공급유형, 자격유형 임대보증금, 임대료
for i, code in tqdm(enumerate(train['단지코드'].unique())):
    temp = train[train['단지코드']==code]
    temp.index = range(temp.shape[0]) # index 재설정
    for col in columns:
        new_train.loc[i, col] = temp.loc[0, col] # Fixed Data
    
    deposit_cost = []
    rental_cost = []

    for col in area_columns:
        area = float(col.split('_')[-1]) # 면적의 숫자를 float형으로 저장
        # '면적_10' 등의 열을 만들고 - 같은 전용면적들 끼리 전용면적별세대수의 합을 Value로 Cell에 입력
        new_train.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()
        
        try:
          deposit_cost.append(temp[temp['전용면적']==area]['임대보증금'][0])
          rental_cost.append(temp[temp['전용면적']==area]['임대료'][0])
        except KeyError as e:
          continue
    
    new_train.loc[i, '등록차량수'] = temp.loc[0, '등록차량수']
    # print(type(deposit_cost[0]))
    new_train.loc[i, '임대보증금'] = int(np.mean(deposit_cost))
    new_train.loc[i, '임대료'] = int(np.mean(rental_cost))

# Test DataFrame Set
# 미사용 열s - 임대건물구분, 공급유형, 자격유형 임대보증금, 임대료, 등록차량수
for i, code in tqdm(enumerate(test['단지코드'].unique())):
    temp = test[test['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_test.loc[i, col] = temp.loc[0, col]

    deposit_cost = []
    rental_cost = []
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_test.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()
        
        try:
          deposit_cost.append(temp[temp['전용면적']==area]['임대보증금'][0])
          rental_cost.append(temp[temp['전용면적']==area]['임대료'][0])
        except KeyError as e:
          continue
          
    new_test.loc[i, '임대보증금'] = int(np.mean(deposit_cost))
    new_test.loc[i, '임대료'] = int(np.mean(rental_cost))

423it [00:08, 50.83it/s]
150it [00:02, 52.24it/s]


# 세종 데이터 나누기

In [186]:
# Train
new_train_sejong = new_train[new_train['지역']==15]
new_train_other = new_train[new_train['지역']!=15]

# Test
new_test_sejong = new_test[new_test['지역']==14]
new_test_other = new_test[new_test['지역']!=14]

# 학습 데이터 나누기

In [187]:
# Train
x_train_sejong = new_train_sejong.iloc[:, :-1]
y_train_sejong = new_train_sejong.iloc[:, -1]

x_train_other = new_train_other.iloc[:, :-1]
y_train_other = new_train_other.iloc[:, -1]

# Test
x_test_sejong = new_test_sejong
x_test_other = new_test_other 

# 학습시킬 데이터 Copy

In [188]:
# Copy하는 이유
# 학습시킬 때 'Code' 열을 지워서 학습시키기 위함
# 학습에 넣을 데이터는 '*_copy'를 넣음
x_train_sejong_copy = x_train_sejong.copy()
x_train_other_copy = x_train_other.copy()

x_test_sejong_copy = x_test_sejong.copy()
x_test_other_copy = x_test_other.copy()

del x_train_sejong_copy['단지코드']
del x_train_other_copy['단지코드']
del x_test_sejong_copy['단지코드']
del x_test_other_copy['단지코드']

# Index 추출

In [189]:
# Index를 추출하는 이유
# submission에서 index로 찾아 값을 넣는게 가장 편함
sejong_idx = x_test_sejong['단지코드'].index
print('세종 Index : ', sejong_idx)
print('세종 개수 : ', len(sejong_idx))

세종 Index :  Int64Index([121, 127, 135], dtype='int64')
세종 개수 :  3


In [190]:
other_idx = x_test_other['단지코드'].index
print('세종 제외 Index : ', other_idx)
print('세종 제외 개수 : ', len(other_idx))

세종 제외 Index :  Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            140, 141, 142, 143, 144, 145, 146, 147, 148, 149],
           dtype='int64', length=147)
세종 제외 개수 :  147


# 학습

In [191]:
forest_sejong = RandomForestRegressor(n_jobs=-1, random_state=42)
forest_other = RandomForestRegressor(n_jobs=-1, random_state=42)

forest_sejong.fit(x_train_sejong_copy, y_train_sejong)
forest_other.fit(x_train_other_copy, y_train_other)

RandomForestRegressor(n_jobs=-1, random_state=42)

# 예측

In [192]:
pred_sejong = forest_sejong.predict(x_test_sejong_copy)
pred_other = forest_other.predict(x_test_other_copy)
#pred

# 데이터프레임으로 편하게 보기

In [193]:
dic = {'세종 INDEX':sejong_idx, '세종 Predict':pred_sejong}
sejong_df = pd.DataFrame(dic)
sejong_df

,세종 INDEX,세종 Predict
0,121,1641.870000
1,127,1618.030000
2,135,1637.330000


In [194]:
dic = {'세종 제외 INDEX':other_idx, '세종 제외 Predict':pred_other}
other_df = pd.DataFrame(dic)
other_df

,세종 제외 INDEX,세종 제외 Predict
0,0,766.040000
1,1,1290.130000
2,2,558.040000
3,3,541.890000
4,4,1166.840000
...,...,...
142,145,251.360000
143,146,497.660000
144,147,35.440000
145,148,467.330000


# Submission에 예측값 입력

In [195]:
submission.loc[sejong_idx, 'num'] = pred_sejong
submission.loc[other_idx, 'num'] = pred_other

# Nan 값 확인

In [196]:
submission.isnull().sum()

code    0
num     0
dtype: int64

In [197]:
submission

,code,num
0,C1072,766.040000
1,C1128,1290.130000
2,C1456,558.040000
3,C1840,541.890000
4,C1332,1166.840000
...,...,...
145,C2456,251.360000
146,C1266,497.660000
147,C2152,35.440000
148,C1267,467.330000


In [198]:
# submission['num'] = pred

In [199]:
# submission.to_csv('baseline.csv', index=False)